In [ ]:
import os
import yaml
import nbimporter
from datetime import datetime, date, timedelta
import yfinance as yf
import time
import random
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,  DateType, TimestampType
from pyspark.sql.functions import to_date, to_timestamp
from lab_database_manager import PgDBManager
from lab_spark import create_spark_session
from lab_schema_manager import SchemaManager

In [ ]:
def fetch_yfinance_record(symbol_date_pairs):
    try:
        symbol, start_date = symbol_date_pairs
        # Fetch stock data using yfinance
        quote = yf.Ticker(symbol)
        end_date = date.today() + timedelta(days=1)
        hist = quote.history(start=start_date, end=end_date)

        # Reset index to include Date as a column and format it
        hist.reset_index(inplace=True)
        hist["Date"] = hist["Date"].dt.date
        
        # limit and stablize the fields of hist
        hist = hist[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
        
        # Add symbol and import_time to each row
        record_list = [
            tuple(row) + (symbol, import_time) for row in hist.itertuples(index=False)
        ]
        random_sleep_time = random.uniform(0.1, 0.9)
        time.sleep(random_sleep_time)

        # print(record_list)
        return record_list

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return []  # Return an empty list on error

In [ ]:
def parallel_fetch_yfinance_record(symbol_date_pairs, record_schema):
    try:
        # Distribute (symbol, start_date) pairs across Spark workers
        record_rdd = spark.sparkContext.parallelize(symbol_date_pairs)

        
        # Fetch data in parallel
        mapped_record_rdd = record_rdd.flatMap(fetch_yfinance_record)

        # Convert RDD to DataFrame
        result_df = spark.createDataFrame(mapped_record_rdd, schema=record_schema)
        # result_df = spark.createDataFrame(mapped_record_rdd)

        # Show or save the results
        # result_df.show()
        return result_df
    except Exception as e:
        print(f"Error paralleling fetch: {e}")
        return spark.createDataFrame([])

In [ ]:
def load_iceberg_raw_eod_yfinance(symbol_date_pairs, iceberg_sink_table, schema_config_file):
    try: 
        schema_manager=SchemaManager(schema_config_file)
        schema_struct_type=schema_manager.get_struct_type("tables", iceberg_sink_table)  
        # print(schema_struct_type)
        
        create_table_script = schema_manager.get_create_table_query("tables", iceberg_sink_table)
        # print(create_table_script)
        spark.sql(create_table_script)

        df_source=parallel_fetch_yfinance_record(symbol_date_pairs, schema_struct_type)        
        # df_source.writeTo(iceberg_sink_table).append()
        df_source.write.mode("overwrite").saveAsTable(iceberg_sink_table)    

        print(f"{iceberg_sink_table} has been loaded")
        
    except Exception as e:
        print(f"Error loading lceberg raw table: {e}")
        

In [ ]:
def load_pg_finalytics(iceberg_source_table, jdbc_url, jdbc_properties, pg_sink_table, pg_truncate_script, pg_merge_script):   
    try:     
        df_source=spark.read.table(iceberg_source_table)          
        finalytics.execute_sql_script(pg_truncate_script)
        
        # Write DataFrame to PostgreSQL
        df_source.write.jdbc(url=jdbc_url, table=pg_sink_table, mode="append", properties=jdbc_properties)        
        finalytics.execute_sql_script(pg_merge_script)
    except Exception as e:
        print(f"Error loading pg finalytics: {e}")        


## Load nessie.raw.stock_eod_yfinance

In [ ]:
# symbol_start_date_pairs = [
#     ('TSLA', '2024-12-20'),
#     ('C', '2024-12-20'),
#     ('SQSP', '2024-12-20'),
# ]

# Get finalytics connetion info
conn_config_file='cfg_connections.yaml'
finalytics=PgDBManager(conn_config_file, 'finalytics')
pg_jdbc_url=finalytics.jdbc_url
pg_jdbc_properties=finalytics.jdbc_properties

# Get symbol_start_date_pairs from finalytics
# query="select symbol, start_date from fin.vw_etl_stock_eod_start_date limit 2"
query="select symbol, start_date from fin.vw_etl_stock_eod_start_date"
symbol_start_date_pairs=finalytics.get_sql_script_result_list(query)
# print(symbol_start_date_pairs)


# Get iceberg table config info
table_schema_config_file='cfg_schemas.yaml'
iceberg_table='nessie.raw.stock_eod_yfinance'

# Set global import_time
import_time = datetime.now().isoformat()


# Create Spark Session
connection_config_file="cfg_connections.yaml"
spark_app_name="raw_yfinance"
spark=create_spark_session(connection_config_file, spark_app_name)
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.raw;")


# Load nessie.raw.stock_eod_yfinance 
load_iceberg_raw_eod_yfinance(symbol_start_date_pairs, iceberg_table, table_schema_config_file)


## Load finalytics.stage.stock_eod_quote_yahoo and merge into fin.stock_eod_quote

In [ ]:
pg_table = "stage.stock_eod_quote_yahoo"  # Replace with the PostgreSQL table name

pg_truncate_script = f"truncate table {pg_table};"

pg_merge_script = "call fin.usp_load_stock_eod();"

load_pg_finalytics(iceberg_table, pg_jdbc_url, pg_jdbc_properties, pg_table,  pg_truncate_script, pg_merge_script)

In [ ]:
spark.sql(f'select count(*) from {iceberg_table}').show()

In [ ]:
spark.sql('select * from nessie.raw.stock_eod_yahoo').show(truncate=False)

In [ ]:
# print(symbol_start_date_pairs)

In [ ]:
# symbol="c"
# start_date ="2024-1-1"
# # Fetch stock data using yfinance
# quote = yf.Ticker(symbol)
# current_date = date.today()
# hist = quote.history(start=start_date, end=current_date)

# # Reset index to include Date as a column and format it
# hist.reset_index(inplace=True)
# print(hist)
# # hist["Date"] = hist["Date"].dt.date
# # print(hist)

# # # limit and stablize the fields of hist
# # hist = hist[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']]

# # # Add symbol and import_time to each row
# # record_list = [
# #     tuple(row) + (symbol, import_time) for row in hist.itertuples(index=False)
# # ]

# # # print(record_list)
# # print(record_list)